<a href="https://www.nvidia.com/dli"><img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/></a>

# Week 2: Identify Nearest Health Facilities

<span style="color:red">
**UPDATE**

Thank you for your analysis. Despite our warning efforts so far, the virus continues to spread rapidly. We want to get infected individuals treatment as quickly as possible, so we need your help to calculate which hospital or clinic is closest to each known infected individual in the population.
</span>

Your goal for this notebook will be to identify the nearest hospital or clinic for each infected person.

## Imports

In [ ]:
%load_ext cudf.pandas
import pandas as pd
import cuml
import cupy as cp

## Load Population Data

Begin by loading the `lat`, `long` and `infected` columns from `'./data/week2.csv'` into a cuDF-accelerated pandas DataFrame called `df`.

In [ ]:
df = pd.read_csv(<<<<FIXME>>>>)
df

## Load Hospital and Clinics Data

For this step, your goal is to create an `all_med` cuDF-accelearted pandas DataFrame that contains the latitudes and longitudes of all the hospitals (data found at `'./data/hospitals.csv'`) and clinics (data found at `'./data/clinics.csv'`).

In [ ]:
hospitals_df = pd.read_csv(<<<<FIXME>>>>)
hospitals_df

In [ ]:
clinics_df = pd.read_csv(<<<<FIXME>>>>)
clinics_df

In [ ]:
all_med = pd.concat([hospitals_df, clinics_df])
all_med

Since we will be using the coordinates of those facilities, keep only those rows that are non-null in both  `Latitude` and `Longitude`.

**Tip**: Reset the index of `all_med` with `.reset_index(drop=True)`. 

In [ ]:
all_med = all_med.dropna(subset=<<<<FIXME>>>>)

all_med = all_med.reset_index(drop=True)

all_med 

## Make Grid Coordinates for Medical Facilities

Provided for you in the next cell (which you can expand by clicking on the "...", and contract again after executing by clicking on the blue left border of the cell) is the lat/long to grid coordinates converter you have used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `all_med` DataFrame you created in the last step.

In [ ]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [ ]:
cupy_lat = cp.asarray(<<<<FIXME>>>>)
cupy_long = cp.asarray(<<<<FIXME>>>>)

all_med['northing'], all_med['easting'] = latlong2osgbgrid_cupy(<<<<FIXME>>>>)
all_med

## Find Closest Hospital or Clinic for Infected

Fit `cuml.NearestNeighbors` with `all_med`'s `northing` and `easting` values, using the named argument `n_neighbors` set to `1`, and save the model as `knn`.

In [ ]:
knn = cuml.<<<<FIXME>>>>(<<<<FIXME>>>>)
knn.fit(<<<<FIXME>>>>)

Save every infected member in `df` into a new dataframe called `infected_df`.

**Tip**: Reset the index of `infected_df` with `.reset_index(drop=True)`. 

In [ ]:
infected_filter = df['infected']==<<<<FIXME>>>>
infected_df = df[infected_filter]
infected_df.reset_index(drop=True)

Create `northing` and `easting` values for `infected_df`.

In [ ]:
cupy_infected_lat = cp.asarray(<<<<FIXME>>>>)
cupy_infected_long = cp.asarray(<<<<FIXME>>>>)

infected_df['northing'], infected_df['easting'] = latlong2osgbgrid_cupy(<<<<FIXME>>>>)
infected_df

Use `knn.kneighbors` with `n_neighbors=1` on `infected_df`'s `northing` and `easting` values. Save the return values in `distances` and `indices`.

In [ ]:
distances, indices = knn.kneighbors(<<<<FIXME>>>>)
infected_df['distance']=distances
infected_df['closest_clinic_hospital']=indices
infected_df

### View Closest Clinic/Hospital ###

`indices`, returned from your use of `knn.kneighbors` immediately above, should map person indices to their closest clinic/hospital indices:

Here you can print an infected individual's coordinates from `infected_df`:

In [ ]:
infected_df.iloc[0] # get the coords of an infected individual (in this case, individual 0)

You should be able to used the mapped index for the nearest facility to see that indeed the nearest facility is at a nearby coordinate:

In [ ]:
all_med.iloc[<<<<FIXME>>>>] # printing the entry for facility 1234 (replace with the index identified as closest to the individual)

## New Patients ##

Use the `knn` model to find the closest clinic/hospital for the below new patients. 

In [ ]:
# DO NOT CHANGE THIS CELL
patients=[
    {
        "name":"John Smith", 
        "northing":462643.2282915547,
        "easting":363859.7580565622
    }, 
        {
        "name":"Greg Brown", 
        "northing":409324.1030472915,
        "easting":464084.5085059871
    }
]

patients_df=pd.DataFrame(patients)

patients_df['distance'], patients_df['closest_clinic_hospital'] = knn.kneighbors(patients_df[['northing', 'easting']])

patients_df['OrganisationID'] = patients_df.apply(lambda x: all_med.loc[x['closest_clinic_hospital'], "OrganisationID"], axis=1)

patients_df

In [ ]:
patients_df.to_json('my_assessment/question_2.json', orient='records')

## Check Submission ##

In [ ]:
!cat my_assessment/question_2.json

**Tip**: Your submission file should contain one line of text, similar to: 

```
[{'name':'John Smith','northing':XXX.XX,'easting':XXX.XX,'closest_clinic_hospital':XX,'OrganisationID':XX},{'name':'Greg Brown','northing':XXX.XX,'easting':XXX.XX,'closest_clinic_hospital':XX,'OrganisationID':XX}]
```

<div align="center"><h2>Please Restart the Kernel</h2></div>

...before moving to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<a href="https://www.nvidia.com/dli"><img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/></a>